## Getting Game info

will be storing seperately from all the game info will both teams in it.

In [ ]:
def getVertTable(currID, driver):
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)
    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]

    headersArr.append(headers)
    columnArr = []
    rows = []
    #strips all rows for each team
    for row in table.find_all('tr')[1:]:
        #td holds info, tr is the stucture
        for td in row.find_all('td'):
            columnArr.append(td.text.strip())
    if columnArr:
        print(columnArr)
        rows.append(columnArr)

    dfStat = pd.DataFrame(rows, columns=np.transpose(headers))
    return dfStat

for link in boxscore_links:
    driver.get(link)
    currDF = getVertTable("game_info", driver)
    break

In [ ]:
def getTeamStats(currID, driver):    
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)

    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]

    headersArr.append(headers)
    columnArr = []
    rows = []
    #strips all rows for each team
    headers = []
    visitor_vals = []
    home_vals = []



    thead = table.find("thead")
    visitor_team = thead.find("th", {"data-stat": "vis_stat"}).text.strip()
    home_team = thead.find("th", {"data-stat": "home_stat"}).text.strip()


    for row in table.find_all('tr'):
        stat_name = row.find('th').text.strip()
        cols = row.find_all('td')
        if len(cols) == 2:
            headers.append(stat_name)
            visitor_vals.append(cols[0].text.strip())
            home_vals.append(cols[1].text.strip())

    # Build DataFrame
    dfStat = pd.DataFrame([visitor_vals, home_vals],
                    index=[visitor_team, home_team],
                    columns=headers)
    return dfStat


for link in boxscore_links:
    driver.get(link)
    currDF = getTeamStats("team_stats", driver)
    break

In [ ]:
def returnEndScores(currID, driver):
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)

    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]

    headersArr.append(headers)
    print(headersArr)
    #strips all rows for each team
    headers = []
    visitor_vals = []
    home_vals = []

    thead = table.find("tbody")

    visitor_team = thead.find_all("td", {"data-stat": "vis_team_score"})[-1].text.strip()
    home_team = thead.find_all("td", {"data-stat": "home_team_score"})[-1].text.strip()
    return visitor_team, home_team


for link in boxscore_links:
    driver.get(link)
    visitorScore, homeScore = returnEndScores("scoring", driver)
    break


### Working code to produce a dataframe for a given team for a given season

In [ ]:
def getGameInfo(currID, driver):
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)
    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]

    headersArr.append(headers)
    columnArr = []
    rows = []
    #strips all rows for each team
    for row in table.find_all('tr')[1:]:
        #td holds info, tr is the stucture
        for td in row.find_all('td'):
            columnArr.append(td.text.strip())
    if columnArr:
        print(columnArr)
        rows.append(columnArr)

    dfStat = pd.DataFrame(rows, columns=np.transpose(headers))
    return dfStat

def returnEndScores(currID, driver):
    homeTeamBool = False
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)

    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]

    headersArr.append(headers)
    print(headersArr)
    #strips all rows for each team
    headers = []
    
    thead = table.find("thead")
    homeTeam = thead.find("th", {"data-stat": "home_team_score"}).text.strip()
    if homeTeam == currTeamAbbriev:
        homeTeamBool = True
        print(f"{currTeamAbbriev} is home")
    else:
        print(f"{currTeamAbbriev} is away")

    tbody = table.find("tbody")
    visitor_team = tbody.find_all("td", {"data-stat": "vis_team_score"})[-1].text.strip()
    home_team = tbody.find_all("td", {"data-stat": "home_team_score"})[-1].text.strip()

    return visitor_team, home_team, homeTeamBool


def getTeamStats(currID, driver):  
    defensiveHeaders = []
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)

    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]
    
    headersArr.append(headers)
    columnArr = []
    rows = []
    #strips all rows for each team
    headers = []
    visitor_vals = []
    home_vals = []



    # thead = table.find("thead")
    # visitor_team = thead.find("th", {"data-stat": "vis_stat"}).text.strip()
    # home_team = thead.find("th", {"data-stat": "home_stat"}).text.strip()


    for row in table.find_all('tr'):
        stat_name = row.find('th').text.strip()
        cols = row.find_all('td')
        if len(cols) == 2:
            headers.append(stat_name)
            visitor_vals.append(cols[0].text.strip())
            home_vals.append(cols[1].text.strip())

    
    visitorScore, HomeScore, homeTeamBool = returnEndScores("scoring", driver)
    
    defensiveHeaders = headers + [f"Opp_{h}" for h in headers]  # now double headers

    if homeTeamBool:
        home_vals = home_vals + visitor_vals
        print(home_vals)
        print(defensiveHeaders)
        dfStat = pd.DataFrame([home_vals],
                        index=[currTeamAbbriev],
                        columns=defensiveHeaders)
        dfStat["HomeTeam"] = True
        dfStat[f"{currTeamAbbriev} Score"] = HomeScore
        dfStat[f"Opp Score"] = visitorScore
    else:
        visitor_vals = visitor_vals + home_vals
        dfStat = pd.DataFrame([visitor_vals],
                        index= [currTeamAbbriev],
                        columns=defensiveHeaders)
        dfStat["HomeTeam"] = False
        dfStat[f"{currTeamAbbriev} Score"] = visitorScore
        dfStat[f"Opp Score"] = HomeScore
    return dfStat